<a href="https://colab.research.google.com/github/joudelshawa/mental-health-ai-app/blob/feature%2Fdata_preprocessing/Topic%20modelling%20with%20NLTK%20and%20LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modelling with SpaCy and NLTK
The following code is from this website: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [14]:
# import libaries
import spacy
import ipynb_path
import nltk 

In [ ]:
# parser = spacy.load("en_core_web_sm")  # Load the trained pipeline for english that is optimized for CPU; create Language object 
from spacy.lang.en import English # import the English class 
parser = English()  # English is a child class of the Language class

def tokenize(text):  # return a list of readable tokens from the English class using default pipeline (what is it tho?)
    lda_tokens = []
    tokens = parser(text)  # applies the loaded pipeline (default pipeline) to the text
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [8]:
nltk.download('wordnet')  # download the WordNet NLTK corpus reader
from nltk.corpus import wordnet as wn

def get_lemma(word):  # use morphy to lemmatize up words not in WordNet
    lemma = wn.morphy(word) 
    if lemma is None:
        return word  
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer

def get_lemma2(word):  # lemmatize words using WordNet's built-in function
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/emily/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
nltk.download('stopwords')  # download stopwords corpus (common words that have little meaning)
en_stop = set(nltk.corpus.stopwords.words('english'))  # set keyword transforms list to iterables

[nltk_data] Downloading package stopwords to /home/emily/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)  # tokenize text 
    tokens = [token for token in tokens if len(token) > 4]  # append all tokens that have len > 4
    tokens = [token for token in tokens if token not in en_stop]  # append all tokens that not in stopwords
    tokens = [get_lemma(token) for token in tokens]  # get base meaning of all words
    return tokens

### Run Preprocessing Notebook
TODO: Try with the goemotions datasets for now; should try to integrate this with other preprocessing code asap

In [8]:
import random
text_data = []
with open('dataset.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

FileNotFoundError: ignored

# Topic Modelling with gensim

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)